In [2]:
// setup
#r "nuget:sequential-reveal-md-tool"

using sequential_reveal_md_tool;

var notebookFileName = "how-to-fall-in-love-with-automated-tests.ipynb";
var styleString = "<link rel=\"stylesheet\" href=\"styles.css\">\n";
var notebook = Notebook.FromFile(notebookFileName);

Installed Packages sequential-reveal-md-tool, 1.1.1

<link rel="stylesheet" href="styles.css">

<div style="position: relative; width: 100%; height: 500px; overflow: hidden;">
  <img src="./media/pawel-czerwinski-VWVO0g9A3rg-unsplash.jpg" alt="Page Break" style="width: 100%; height: auto;">
  <div style="
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(to bottom, rgba(0, 0, 0, 0) 60%, #1F1F1F 100%);
    pointer-events: none;">
  </div>
</div>

# How to fall in love with automated tests! 😍

<div class="standardCellBottomSpace"></div>

<link rel="stylesheet" href="styles.css">

# E-2-e Component Tests

### How to get the maximum mileage from a minimal number of tests🚀

# ...

# Integration Tests

### how _not_ to hate them! ❤️

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Let’s start with a scenario…

animated diagram here?

<div style="display:block; height: 30rem;"></div>

<!-- End-to-end Component Tests - What? -->
<link rel="stylesheet" href="styles.css">

# End-to-end Component Tests - What?

## Caveat: opinions - none are right or wrong!!

- Unit is as large as possible - multiple executables!
- Consolidation (one project for multiple components)
- Test as much of the surface area as possible

trophy diagram

<div style="display:block; height: 30rem;"></div>


In [9]:
//Notebook.FromFile("how-to-fall-in-love-with-automated-tests.ipynb").DisplayNextSequentialSectionFromPrecedingMarkdownCellUsingStyleString("sr1", "<link rel=\"stylesheet\" href=\"styles.css\">\n");
Notebook.FromFile(notebookFileName).DisplayNextSequentialSectionFromPrecedingMarkdownCell("sr1");

<!-- End-to-end Component Tests - What? -->

<link rel="stylesheet" href="styles.css">

# End-to-end Component Tests - What?



<link rel="stylesheet" href="styles.css">

# End-to-end Component Tests - Why?

- Test the behaviour not the impl
- Blackbox, refactor away😁
- Almost as good as running locally
- Writing tests is _not_ fun, building a test framework _can_ be fun!

<div style="display:block; height: 30rem;"></div>


<link rel="stylesheet" href="styles.css">

# How? #1 Make a nice framework😊

## Given, When & Then or Arrange, Act & Assert etc...

```csharp
[Fact]
public void Return_a_WeatherReport_given_valid_region_and_date()
{
    var (given, when, then) = testFixture.SetupHelpers();

    given.WeHaveAWeatherReportRequest("bristol", DateTime.Now, out var apiRequest)
        .And.TheServersAreStarted();

    when.WeSendTheMessageToTheApi(apiRequest, out var response);

    then.TheResponseCodeShouldBe(response, HttpStatusCode.OK)
        .And.TheBodyShouldNotBeEmpty<WeatherReportResponse>(response, 
            x => x.Summary.Should().NotBeEmpty());
}
```

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# How? #2 Create a test host factory for each executable app

## Microsoft.AspNetCore.Mvc.Testing package

```csharp
public class ApiWebApplicationFactory : WebApplicationFactory<API.Program>
{
    public HttpClient? HttpClient;

    public readonly Mock<ILogger> MockLogger = new();
    public readonly Mock<HttpMessageHandler> MockWeatherModelingServiceHttpMessageHandler = new();

    public Func<EventRepositoryInMemory>? SetSharedEventRepository = null;

    // Using CreateHost here instead of ConfigureWebHost because CreateHost adds config just after 
    // WebApplication.CreateBuilder(args) is called
    // whereas ConfigureWebHost is called too late just before builder.Build() is called.
    protected override IHost CreateHost(IHostBuilder builder)
    {
        Environment.SetEnvironmentVariable("WeatherModelingServiceOptions__BaseUrl", Constants.WeatherModelingServiceBaseUrl);
        Environment.SetEnvironmentVariable("WeatherModelingServiceOptions__MaxRetryCount", "3");

        builder
            .ConfigureServices(services =>
            {
                var loggerFactory = new Mock<ILoggerFactory>();
                loggerFactory.Setup(x => x.CreateLogger(It.IsAny<string>())).Returns(MockLogger.Object);
                services.AddSingleton(loggerFactory.Object);

                services.AddHttpClient(typeof(IWeatherModelingServiceClient).FullName!, client => client.BaseAddress = 
                    new Uri(Constants.WeatherModelingServiceBaseUrl))
                        .ConfigurePrimaryHttpMessageHandler(() => MockWeatherModelingServiceHttpMessageHandler.Object);
                    
```

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# How? #3 Single Test Fixture

```csharp
public class ComponentTestFixture : IDisposable
{
    private string phase = "";

    public readonly ApiWebApplicationFactory ApiFactory;
    public readonly EventListenerWebApplicationFactory EventListenerFactory;
    public readonly NotificationServiceWebApplicationFactory NotificationServiceFactory;
    
    public readonly MockServiceBus MockServiceBus;

    public EventRepositoryInMemory EventRepositoryInMemory = new();

    public ComponentTestFixture()
    {
        ApiFactory = new() { SetSharedEventRepository = () => EventRepositoryInMemory };
        EventListenerFactory = new(this) { SetSharedEventRepository = () => EventRepositoryInMemory };
        NotificationServiceFactory = new();

        MockServiceBus = new MockServiceBus();
        MockServiceBus.AddSenderFor<DummyIntegrationEvent>();
        MockServiceBus.AddProcessorFor<ModelingDataAcceptedIntegrationEvent>();
        MockServiceBus.AddProcessorFor<ModelingDataRejectedIntegrationEvent>();
        MockServiceBus.AddProcessorFor<ModelUpdatedIntegrationEvent>();
    }

    public void Dispose()...

    public (Given given, When when, Then then) SetupHelpers()
    {
        return (new Given(this), new When(this), new Then(this));
    }

    public void SetPhase(string newPhase) => phase = newPhase;
    public string CurrentPhase => string.IsNullOrWhiteSpace(phase) ? string.Empty : $"In phase {phase}, ";
}
```

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# How? #4 Testable Service Bus Processor😎

```csharp
public class TestableServiceBusProcessor(string dummyQueueName) : ServiceBusProcessor
{
    public string DummyQueueName { get; } = dummyQueueName;
    public List<TestableProcessMessageEventArgs> MessageDeliveryAttempts = [];
```
---
⬇️

```csharp
    public override Task StartProcessingAsync(CancellationToken cancellationToken = default)
    {
        return Task.CompletedTask;
    }
```
---
⬇️

```csharp
    public async Task SendMessage<T>(
        T request, int deliveryCount = 1, Dictionary<string, object>? applicationProperties = null) 
        where T : class
    {
        var args = CreateMessageArgs(request, deliveryCount, applicationProperties);
        MessageDeliveryAttempts.Add((TestableProcessMessageEventArgs)args);
        await base.OnProcessMessageAsync(args);
    }
```
---
⬇️

```csharp
    public static ProcessMessageEventArgs CreateMessageArgs<T>(
        T payload, int deliveryCount = 1, Dictionary<string, object>? applicationProperties = null) 
        where T : class
    {
        var payloadJson = JsonSerializer.Serialize(payload, GlobalJsonSerialiserSettings.Default);

        var correlationId = Guid.NewGuid().ToString();
        applicationProperties ??= new Dictionary<string, object>
        {
            { "origin", "ComponentTests" }
        };

        var message = ServiceBusModelFactory.ServiceBusReceivedMessage(
            body: BinaryData.FromString(payloadJson),
            correlationId: correlationId,
            properties: applicationProperties,
            deliveryCount: deliveryCount);

        var args = new TestableProcessMessageEventArgs(message);

        return args;
    }
}
```
---
⬇️

```csharp
public class TestableProcessMessageEventArgs(ServiceBusReceivedMessage message) 
    : ProcessMessageEventArgs(message, null, CancellationToken.None)
{
    public bool WasCompleted;
    public bool WasDeadLettered;
    public bool WasAbandoned;
    public DateTime Created = DateTime.UtcNow;
    public string DeadLetterReason = string.Empty;

    public override Task CompleteMessageAsync(ServiceBusReceivedMessage message,
        CancellationToken cancellationToken = new())
    {
        WasCompleted = true;
        return Task.CompletedTask;
    }
```

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# How? #5 Mock Service Bus Sender 🚌

## ServiceBusClient can be Mocked using your favourite mocking framework

```csharp
public Then AMessageWasSent(Mock<ServiceBusSender> senderMock, int times = 1)
{
    senderMock.Verify(x => x.SendMessageAsync(
        It.IsAny<ServiceBusMessage>(), It.IsAny<CancellationToken>()), Times.Exactly(times));

    return this;
}
```

## If one service sends a message to another, use Callback() or equivalent

```csharp
public class MockServiceBus
{
    private readonly Dictionary<Type, TestableServiceBusProcessor> processors = new();
    private readonly Dictionary<Type, Mock<ServiceBusSender>> mockSenders = new();

    public void WireUpSendersAndProcessors(IServiceCollection services) {...}

    // plus methods for adding and getting Processors ...

    public void ClearDeliveryAttemptsOnAllProcessors() {...}

    // plus methods for adding and getting Senders ...

    public void ClearAllInvocationsOnAllSenders() {...}

    public void MessagesSentToSenderWillBeReceivedOnCorrespondingProcessor<TMessageType>() where TMessageType : class {...}

    public void MessagesSentToSendersWillBeReceivedOnCorrespondingProcessors() {...}
}
```

## There is also the official Azure service Bus Emulator🤔

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# How? #6 Re-wire Http Clients if needed

```csharp
public class EventListenerWebApplicationFactory : WebApplicationFactory<EventListener.Program>
{    
    private readonly CustomHttpClientFactory customHttpClientFactory = new();
```

``` csharp
    protected override void ConfigureWebHost(IWebHostBuilder builder)
    {
        base.ConfigureWebHost(builder);
        builder.ConfigureServices(services =>
        {
            // Replace standard IHttpClientFactory impl with custom one with any added HTTP clients.
            services.AddSingleton<IHttpClientFactory>(customHttpClientFactory);
        });
    }
```

```csharp
    public void ClearHttpClients() => customHttpClientFactory.HttpClients.Clear();

    public void AddHttpClient(string clientName, HttpClient client)
    {
        if (customHttpClientFactory.HttpClients.TryAdd(clientName, client) == false)
        {
            throw new InvalidOperationException($"HttpClient with name {clientName} is already added");
        }
    }
```

```csharp
}
```

---

```csharp
public class CustomHttpClientFactory() : IHttpClientFactory
{
    public Dictionary<string, HttpClient> HttpClients = [];

    public HttpClient CreateClient(string name) =>
        HttpClients.GetValueOrDefault(name)
        ?? throw new InvalidOperationException($"HTTP client is not found for client with name {name}");
}
```

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# How? #7 Database connections

- Replace db connection in IoC container with Mock backed by in-memory collections
- In-memory db context for EF Core
- Or use a real db connection with something like CSharpSqlTests

animated diagram?

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Demo time!

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Pros and Cons

✔️ The more reusable the code is, the more care and love is justified

✔️ Test entire e2e flows in addition to individual phases 

✔️ Detect config and IoC registration issues

✔️ Not locked into any particular tooling

<div style="display:block; height: 5rem;"></div>

✖️ TDD is possible but is definitely a bit harder getting started

✖️ Timing and async

✖️ Still need to run locally to prove integrations/mocked behaviours work as expected etc

<div style="display:block; height: 30rem;"></div>


<link rel="stylesheet" href="styles.css">
<style> 
h1, h3 {
  text-align: center;
  font-weight: bold;
}
</style>

# Integration Tests

### how _not_ to hate them! ❤️

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# The problem with integration tests…

- They tend to be the last task on a story… 😒
- Feedback loop: `change -> CI build -> release -> tests` is way too long 🥱
- They expose how difficult it is to run things locally 😬

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# The problem with local dev ex…

- Heavily dependant on Compute platform
- Too many options!
- Large differences between local and real running in an environment


<div style="display:block; height: 30rem;"></div>


<link rel="stylesheet" href="styles.css">

# Specific issues…

Bacs! 🤯

1) Config
    - Too many options!
    - Managing git changes - don't check in secrets!

2) Service Bus Topics, Subscriptions & Queues
    - sharing a team env namespace, with prefixing
    - manual queue creation in Azure portal😒


<div style="display:block; height: 30rem;"></div>


<link rel="stylesheet" href="styles.css">

# Differences between local and real env/pipeline

- ManagedIdentity from within containers
    ```
    #IF DEBUG 😒
    VS != AKS!
    ```
- Communication with pods without Ingress etc
    ```
    No local APIM
    Different base urls per service rather than one
    ```


<div style="display:block; height: 30rem;"></div>


<link rel="stylesheet" href="styles.css">

# In an ideal world…

We would:

- Clone a repo
- Run non-integration tests, all pass first time
- Run a single local deploy script/command
- Run integration tests, all pass first time
- From there on, hit [F5] and everything runs nicely😁

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Closing gaps between local and env running…

e.g for AKS via Helm charts and Skaffold...

- Use same tooling as release where possible?
- Your app sees environment variables and secrets as config providers, why not just use system environment variables locally?
- External services probably all exist in your team env?

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# 1) Config, potential fix…

Aim for fewest types of config:

- Static config in appSettings.json, 
- everything else in environment variables

**Once agreed, keep it consistent!🚀**

Manage environment variables with Powershell…

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Use Powershell…

```
Powershell script (specify team/env)
-> az login
-> set active subscription
-> fetch ASB connection string & namespace
-> fetch APIM base endpoint & keys for specific products
-> write into env vars
-> restart Visual Studio!
```
hint: copilot chat is very good at creating things like this! 🤖

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Azure Service Bus potential fix…

A ServiceBus Fixture likely already exists?

It should create topics, subscriptions and forwarding queues?

Prefix with machine name to share single namespace

When running locally just don’t delete them at the end!

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Demo!

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# ToDo...

- Prefix env variables for different apps in script
- Use of shared local running helm charts?
- Use of `DefaultCredential` for authenticating via MI from inside containers
- Improve local database setup and deployment story
- Investigate bridge-to-kubernetes

<div style="display:block; height: 30rem;"></div>

<link rel="stylesheet" href="styles.css">

# Pros and Cons

✔️ Slightly closer to the ideal scenario

✔️ consistent results across everyone's machines

✖️ scripts to maintain

<div style="display:block; height: 30rem;"></div>

In [ ]:
// CSharp sample code
var now = TimeProvider.System.GetUtcNow();
Console.WriteLine($"Hello world! @ {now}");